### Import Libraries

In [23]:
import yfinance as yf
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pypfopt import plotting
import cvxpy 
import pypfopt.plotting as plotting
from datetime import datetime, timedelta
import warnings
# warnings.filterwarnings("ignore")

In [24]:
#USCAD currency, gold, S&P 500, long term tradable treasury bond, bitcoin
# tickers = ['USDCAD=X', 'GLD', '^GSPC', 'TLT', 'BTC-USD']
tickers = ['GLD', '^GSPC', 'BTC-USD']
end_date = datetime.today() - timedelta(days=1)

start_date = end_date - timedelta(days=5*365)
adj_close_df = pd.DataFrame()

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=False)
    adj_close_df[ticker] = data['Adj Close']

print(adj_close_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

                   GLD        ^GSPC        BTC-USD
Date                                              
2020-05-20  164.649994  2971.610107    9522.981445
2020-05-21  162.250000  2948.510010    9081.761719
2020-05-22  163.210007  2955.449951    9182.577148
2020-05-26  160.889999  2991.770020    8835.052734
2020-05-27  161.179993  3036.129883    9181.017578
...                ...          ...            ...
2025-05-13  299.459991  5886.549805  104169.812500
2025-05-14  293.160004  5892.580078  103539.414062
2025-05-15  297.839996  5916.930176  103744.640625
2025-05-16  294.239990  5958.379883  103489.289062
2025-05-19  298.029999  5963.600098  105606.179688

[1256 rows x 3 columns]


### Calculate Log Returns

In [25]:
#log of row divided by previous row
log_returns = np.log(adj_close_df / adj_close_df.shift(1))
log_returns = log_returns.dropna()
log_returns

,GLD,^GSPC,BTC-USD
Date,,,
2020-05-21,-0.014684,-0.007804,-0.047440
2020-05-22,0.005899,0.002351,0.011040
2020-05-26,-0.014317,0.012214,-0.038581
2020-05-27,0.001801,0.014718,0.038411
2020-05-28,0.003345,-0.002110,0.036861
...,...,...,...
2025-05-13,0.004250,0.007222,0.013111
2025-05-14,-0.021262,0.001024,-0.006070
2025-05-15,0.015838,0.004124,0.001980


### Forecast Expected Returns

In [26]:
mu2 = expected_returns.mean_historical_return(adj_close_df)
print(mu2)

GLD        0.126536
^GSPC      0.150123
BTC-USD    0.621123
dtype: float64


### Covariance Matrix

In [27]:
cov_matrix = risk_models.sample_cov(adj_close_df)
print(cov_matrix)

              GLD     ^GSPC   BTC-USD
GLD      0.023456  0.003945  0.009755
^GSPC    0.003945  0.031599  0.037454
BTC-USD  0.009755  0.037454  0.370019


### Efficient Frontier

In [28]:
ef = EfficientFrontier(mu2, cov_matrix)
ef.max_sharpe()


OrderedDict([('GLD', 0.5267130313582468),
             ('^GSPC', 0.318881668317748),
             ('BTC-USD', 0.1544053003240052)])

In [29]:
ef.portfolio_performance(verbose=True)

Expected annual return: 21.0%
Annual volatility: 15.9%
Sharpe Ratio: 1.33


(np.float64(0.21042423309834063),
 np.float64(0.15856371388309912),
 np.float64(1.3270642314386987))